<span type="title">单周期处理器</span> | <span type="update">2018-07-15</span> | <span type="version">2</span>

<span type="intro"><p type="card-text">本章主要讲解单周期处理器。</p></span>

# 处理器的设计步骤

- 分析指令，得出对数据通路的需求
- 为数据通路选择合适的组件
- 连接各部分组件以建立数据通路
- 分析每条指令的实现，确定控制信号
- 控制信号的整合，形成完整的控制逻辑

# 1. 数据通路和组件构成

## 1.1 指令及需求

本章节主要实现六条指令的MIPS：

- addu rd,rs,rt
- subu rd,rs,rt
- ori rt,rs,imm16
- lw rt,imm16(rs)
- sw rt,imm16(rs)
- beq rs,rt,imm16

这六条指令对应了储存器数据的存储、算术加减和立即数逻辑或运算，还有一条条件分支语句。

根据我们的需求——实现六条MIPS语句，不难得出，我们需要以下部件：

- 存储器：一个只读的指令存储器，地址和数据均为32位，一个可读写的数据存储器，地址和数据均为32位。（对应CPU中的指令和数据高速缓存，而不是DRAM内存）。
- 程序计数器：32位的寄存器，用来保存下一条程序指令地址。
- 立即数扩展器：将16位立即数扩展为32位，支持零扩展和符号扩展
- 寄存器堆：每个寄存去32bit，共32个，其中rs和rt需要支持读操作，rt或rd需要支持写操作（两读一写寄存器堆）
- ALU：可以支持加减、或、相等的比较，能够操作2个32位数，其来自寄存器或者扩展后的立即数。




## 1.2 存储组件：寄存器堆

寄存器堆的组成如下所示，其包含32bit×32个寄存器，其中busA,B为数据输出信号，32bit宽，bugW为数据输入信号，32bit宽，读写控制由Rw，Ra，Rb控制，其中Rw控制写入，当WriteEnable为True并且clk到来的时候控制busW内容存入该寄存器。Ra和Rb控制busA，B的读操作。

此处的读指的是，数据由寄存器输出到别的组件，别的组件在读。此处的写指的是，别的组件通过busW写入数据到RegFile。

注意，寄存器堆的读操作不受时钟控制，但是写入操作受时钟控制。

![](w5p1.png)

## 1.3 存储组件：存储器

存储器包含一个输入接口，一个输出接口，均为32bit，其读写控制由32位的Address控制，此信号指定一个存储单元，将其内容送到数据输入处。

Write Enable 写使能信号，在时钟信号上升沿，如果写使能信号为1（有效），则将数据输入信号的内容存储在内部。

注意，同寄存器堆，从储存器进行的读操作不受时钟控制。

![](w5p2.png)

## 1.4 指令存取单元 IFU

所有的指令经过抽象化，有一个共同的部分，就是从PC寄存器取出指令地址，经过存储器取出指令内容，然后PC寄存器转向下一条指令地址或者目标地址的步骤，如下图所示：

![](w5p3.png)

其中的nPC_sel指的是来自外部的特殊信号，控制PC打断当前循环，取特定地址的指令，类似于goto。这个所有指令的共同部分，称之为IFU，Instruction
 Fetch Unit。

## 1.5 形成完整数据通路

![](w5p4.png)

如上图所示，是完整的数据通路。

对于加法/减法而言： `addu rd,rs,rt; R[rd] = R[rs] op R[rt]`

组件：RegFile寄存器堆（Ra接入rs，Rb接入rt，Rw接入rd），ALU

信号：ALUCtr 控制加法或者减法，RegWr 控制在计算完毕后，写入开关为开

对于逻辑运算而言： `ori rt,rs,imm16; R[rt] = R[rs] op ZeroExt[imm16] `

组件：RegFile加强版 (Rw = rt，Ra = rs), ALU, Extender。Rb不用输入，并且busB需要被截断，因此有四个信号：

信号：ALUCtr 控制运算信号，ExtOp 控制立即数扩张信号，ALUSrc 控制busB or imm16的信号，RegDst 选择何者被写入数据的信号，RegWr 控制写入信号。

对于访存指令而言： `R[rt] = Mem[R[rs]+SignExt[imm16]]; lw rt,imm16(rs)` 

组件：RegFile加强版，ALU，Extender，Data Memory

信号：ALUCtr，ALUSrc，RegDst（存储在rt中而非rd），RegWr写回数据控制信号，ExtOp 立即数的符号扩展信号，MemtoReg ALU经过计算后到内存中取地址，控制写回到busW中的信号。

对于Store而言，`Mem[R[rs]+SignExt[imm16]] = R[rt]` 

需要多一个MemWr的控制写入到内存中的信号，方便从busB直接写入数据到Data中。

# 加法指令的操作步骤

对于加法指令而言，其步骤有三：

- 从PC获取指令：MEM[PC]
- 执行加法：R[rd]=R[rs]+R[rt]
- 计算下一条指令地址：PC = PC + 4

![](w5p5.png)

如上所示，是第一步，其中从clk的第一个上升沿开始，到红线为止，这一步已经执行完毕，其消耗时间小于半个时钟周期。

然后在此刻执行加法，时钟后半个周期的一半时，如下所示，IFU传出的数据被放置到RegFile，ALUCtr被指向ADD，同时设置好了其余指令。这个步骤主要被用作信号设置。

![](w5p6.png)

当时钟周期快要结束的时候，信号在如图红线部分传递，正在执行这个加法过程。

![](w5p7.png)

在上步已经设置了nPC_sel = 0（+4） 的信号，因此在这个周期末尾，执行PC+4的运算并且执行下一条指令。

![](w5p8.png)

这就是一个完整的加法运算周期，正好使用了一个周期。

# 逻辑或指令的运算步骤

`ori rt, rs, imm16`
- MEM[PC] 从指令存储器中取回指令
- `R[rt]=R[rs] | ZeroExt[imm16]` 指令指定的操作
- PC=PC + 4 计算下一条指令的地址

![](w5p9.png)

如上图所示，和加法类似，设置好信号后，先进行IFU操作，得到指令，imm16扩容，同时接受busA输入，在ALU进行OR运算，结果写会到rt寄存器，其中Memory读写始终关闭，busB也关闭。

# 读数据指令的操作步骤

`lw rt, imm16(rs)`
- MEM[PC] 从指令存储器中取回指令
- `R[rt]=DataMemory{R[rs]+SignExt[imm16]}` 指令指定的操作
- PC=PC + 4 计算下一条指令的地址

![](w5p10.png)

如上图所示，由IFU得到指令，然后将地址内容放入rs，rs被吓一个扩充过的立即数经过ALU相加得到数值，数值没有被写回寄存器，而是经过Adr接口获取到内容后经过MemtoReg控制写入bugW，返回rt寄存器。

# 存放数据指令的操作步骤

`sw rt, imm16(rs)`
- MEM[PC] 从指令存储器中取回指令
- `DataMemory{R[rs]+SignExt[imm16]}=R[rt]` 指令指定的操作
- PC=PC + 4 计算下一条指令的地址

同上，经过了以下步骤：

![](w5p11.png)

# 分支指令的判断步骤

`beq rs, rt, imm16`
- MEM[PC] 从指令存储器中取回指令
- `if(R[rs]-R[rt]==0)` 判断转移条件是否成立
- `then PC = PC + 4 + SignExt[imm16]*4 ;`
`else PC = PC + 4 ;`
计算下一条指令的地址

如下所示：

![](w5p12.png)

注意，这一步仅仅是在执行判断，ALUshu结果经过zero判断输入IFU，